In [1]:
from qdrant_client import QdrantClient, models

/home/berk/finreg-navigator/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = QdrantClient(url="http://localhost:6333")

In [3]:
collection_name = "bank_compliance_v1"

client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
            hnsw_config=models.HnswConfigDiff(
                m=8,
                ef_construct=100
            )
        )
    },
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    },
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=1000
    )
)

client.create_payload_index(collection_name, "file_name", models.PayloadSchemaType.KEYWORD)
client.create_payload_index(collection_name, "page_label", models.PayloadSchemaType.INTEGER)
client.create_payload_index(collection_name, "article_no", models.PayloadSchemaType.KEYWORD)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
import os
from llama_index.core import SimpleDirectoryReader

def banking_metadata_extractor(file_path):
    """
    Her dosya okunduğunda dosya bazlı sabit metadata'ları ekler.
    LlamaIndex sayfa numaralarını (page_label) otomatik olarak dökümana ekleyecektir.
    """
    file_name = os.path.basename(file_path)
    
    # Dosya ismine göre döküman başlığı atama (Enterprise ETL mantığı)
    doc_titles = {
        "BBSEBH.pdf": "Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik",
        "sir_saklama_yukumlulugu.pdf": "Sır Niteliğindeki Bilgilerin Paylaşılması Hakkında Yönetmelik"
    }
    
    return {
        "file_name": file_name,
        "document_title": doc_titles.get(file_name, "Bilinmeyen Mevzuat"),
        "file_path": file_path,
        "category": "Banking Regulation",
        "ingestion_date": "2024-05-22"
    }

# Veriyi Yükleme
reader = SimpleDirectoryReader(
    input_dir="./data", 
    file_metadata=banking_metadata_extractor
)
documents = reader.load_data()

print(documents[0].metadata)

{'page_label': '1', 'file_name': 'BBSEBH.pdf', 'document_title': 'Bankaların Bilgi Sistemleri ve Elektronik Bankacılık Hizmetleri Hakkında Yönetmelik', 'file_path': '/home/berk/finreg-navigator/data/BBSEBH.pdf', 'category': 'Banking Regulation', 'ingestion_date': '2024-05-22'}


In [5]:
from llama_index.core.node_parser import HierarchicalNodeParser

node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048,512,128],
    chunk_overlap=20
)

nodes = node_parser.get_nodes_from_documents(documents)

In [6]:
import re

def enrich_metadata_with_article(nodes):
    current_article = "Bilinmiyor"
    for node in nodes:
        # Sadece hiyerarşinin en üstündeki (büyük) node'larda madde ara
        # veya her node'da ara ama bulamazsan bir önceki maddeyi kullan
        match = re.search(r"MADDE\s+(\d+)", node.text)
        if match:
            current_article = f"Madde {match.group(1)}"
            node.metadata["article_no"] = current_article
        else:
            # Eğer bu küçük parça madde no içermiyorsa, 
            # ait olduğu dökümandaki son bulunan maddeyi ata
            node.metadata["article_no"] = current_article
    return nodes

enriched_nodes = enrich_metadata_with_article(nodes)

In [7]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Banking_RAG_Ingestion")

with mlflow.start_run(run_name="Initial_Ingestion_Hybrid"):
    # Parametreleri logla
    mlflow.log_param("chunk_sizes", "[2048, 512, 128]")
    mlflow.log_param("vector_store", "Qdrant")
    mlflow.log_param("collection_name", "banking_compliance_v1")
    
    # Node sayısını logla
    mlflow.log_metric("total_nodes", len(enriched_nodes))
    
    print(f"Toplam {len(enriched_nodes)} node oluşturuldu ve MLflow'a loglandı.")

Toplam 1751 node oluşturuldu ve MLflow'a loglandı.
🏃 View run Initial_Ingestion_Hybrid at: http://localhost:5000/#/experiments/1/runs/bfe589ea61ff44628cfd7bd874cfcdb5
🧪 View experiment at: http://localhost:5000/#/experiments/1


In [8]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. Embedding Modelini Tanımla (Dense)
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Qdrant Store Yapılandırması
vector_store = QdrantVectorStore(
    collection_name="bank_compliance_v1",
    client=client,
    enable_hybrid=True,
    batch_size=64,
    dense_vector_name="dense",
    sparse_vector_name="sparse"
)

# 3. Indexleme İşlemi
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(
    enriched_nodes, 
    storage_context=storage_context,
    embed_model=embed_model
)

print("İndeksleme başarıyla tamamlandı!")

2026-02-11 19:11:33,293 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2026-02-11 19:11:33,577 - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-11 19:11:33,623 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
2026-02-11 19:11:33,772 - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-11 19:11:33,820 - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json "HTTP/1.1 200 OK"
2026-02-11 19:11:33,967 - INFO - HTTP Request: HEAD https://huggingface.co/sent

İndeksleme başarıyla tamamlandı!


In [9]:
# Geri getiriciyi (Retriever) yapılandır
# similarity_top_k=3: En alakalı 3 parçayı getir
retriever = index.as_retriever(similarity_top_k=3)

# Test Sorgusu
query = "Açık bankacılık servisleri nelerdir ve kimler erişebilir?"
retrieved_nodes = retriever.retrieve(query)

# Sonuçları İnceleme
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Sonuç {i+1} (Skor: {node.score:.4f}) ---")
    print(f"Kaynak: {node.metadata.get('file_name')} | Sayfa: {node.metadata.get('page_label')}")
    print(f"Madde: {node.metadata.get('article_no', 'Belirtilmemiş')}")
    print(f"İçerik Özeti: {node.text[:200]}...")

2026-02-11 19:15:53,709 - INFO - HTTP Request: POST http://localhost:6333/collections/bank_compliance_v1/points/query/batch "HTTP/1.1 200 OK"



--- Sonuç 1 (Skor: 0.7274) ---
Kaynak: BBSEBH.pdf | Sayfa: 1
Madde: Madde 1
İçerik Özeti: n) Güvenlik duvarı: Farklı güvenlik seviyelerine sahip ağlar veya ağa bağlı cihazlar arasındaki trafik akış kontrolünü sağlayan donanım ya da
yazılımları,
o) Hassas veri: Kimlik doğrulamada kullanılan...

--- Sonuç 2 (Skor: 0.7257) ---
Kaynak: BBSEBH.pdf | Sayfa: 2
Madde: Madde 4
İçerik Özeti: bilgi sistemlerinden sorumlu üst düzey yönetici ile bankanın ilgili iş
birimlerinden üst düzey yöneticilerin bu komiteye üye olması esastır....

--- Sonuç 3 (Skor: 0.7224) ---
Kaynak: sir_saklama_yukumlulugu.pdf | Sayfa: 1
Madde: Madde 1
İçerik Özeti: c) Ana sermaye: 5/9/2013 tarihli ve 28756 sayılı Resmî Gazete’de yayımlanan Bankaların Özkaynaklarına
İlişkin Yönetmelikte belirlenen usul ve esaslar çerçevesinde hesaplanacak ana sermayeyi,
ç) Anonim...
